In [6]:
## Librerias
import numpy as np
import scipy
from scipy import linalg
from numpy.linalg import inv

In [7]:
### La funcion descomponer recibe una matriz(lista de listas) de 3x4
### Devuelve las matrices K[3x3], R[3X3], t[3x1] 
def descomponer(listaDeListas):
    
    ## Convertimos lista de listas a matriz de Numpy
    matriz  = np.array(listaDeListas)
    print("\n Matriz M \n -------------")
    print(matriz)
    
    
    ## Calculamos S
    s = matriz[:,0:3]
    print("\n Matriz S \n -------------")
    print(s)
    
    
    ## Calculamos K y R utilizando rq de la libreria numpy.linalg
    k, r = linalg.rq(s)
    print("\n Matriz K \n -------------")
    print(k)
    
    print("\n Matriz R \n -------------")
    print(r)
    
    
    ## Cambiamos los signos de las columnas de K (cuando el elmento de su diagonal es menor a 0)
    ## Cambiamos los signos de las filas de R (cuando el elemento de la diagonal en K es menor a 0)
    for j in [0,1,2]:    
        if k[j,j] < 0:
            r[j,:] = -r[j,:]
            k[:,j] = -k[:,j]

    print("\n Matriz K sin negativos \n -------------")
    print(k)
    print("\n Matriz R sin negativos \n -------------")
    print(r)

    
    ## Calculamos w = k.t = tercer columna de M
    w = matriz[:,3]
    
    k_inv = inv(k)
    print("\n Matriz K invertida \n -------------")
    print(k_inv)
    
    t = k_inv.dot(w)
    print("\n Matriz t \n -------------")
    print(t)
    
    return k,r,t

In [8]:
## Prueba de la funcion descomponer

# Se crean matrices de k,r y t
k = [[1,2,0],[0,3,4],[0,0,2]]
r = [[0,0,1],[0,1,0],[1,0,0]]
t = [[1],[0],[2]]

rt = [[0,0,1,1],[0,1,0,0],[1,0,0,2]]

# Se crean las matrices en numpy
k  = np.array(k)
r  = np.array(r)
t  = np.array(t)
rt  = np.array(rt)

# Se calcula M
m = k.dot(rt)
print(m)

# Se descompone y se obtiene R, K y T para verificar si es igual a los creados anteriormente
k_calculada,r_calculada,t_calculada =  descomponer(m)


[[0 2 1 1]
 [4 3 0 8]
 [2 0 0 4]]

 Matriz M 
 -------------
[[0 2 1 1]
 [4 3 0 8]
 [2 0 0 4]]

 Matriz S 
 -------------
[[0 2 1]
 [4 3 0]
 [2 0 0]]

 Matriz K 
 -------------
[[-1.  2.  0.]
 [ 0.  3. -4.]
 [ 0.  0. -2.]]

 Matriz R 
 -------------
[[ 0.  0. -1.]
 [ 0.  1.  0.]
 [-1. -0.  0.]]

 Matriz K sin negativos 
 -------------
[[ 1.  2. -0.]
 [-0.  3.  4.]
 [-0.  0.  2.]]

 Matriz R sin negativos 
 -------------
[[-0. -0.  1.]
 [ 0.  1.  0.]
 [ 1.  0. -0.]]

 Matriz K invertida 
 -------------
[[ 1.         -0.66666667  1.33333333]
 [ 0.          0.33333333 -0.66666667]
 [ 0.          0.          0.5       ]]

 Matriz t 
 -------------
[ 1.  0.  2.]


In [76]:
### La funcion proyectar recibe una matriz con la posicion de dimensiones 3xN
### una matriz de calibracion de 3x4 y una matriz de 2x1 con el tamaño del sensor
### Devuelve la matriz proyeccion de [2xN]
def proyectar(pos, matCalib, tamanioSensor):
    
    ## Primero utilizamos la ecuacion (u,v)^T Mc^T X = (Ma,Mb)^T X
      # Queremos calcular (u,v)
        
    ## Pasamos los parametros a matrices de numpy
    pos  = np.array(pos)
    matCalib  = np.array(matCalib)
    tamanioSensor  = np.array(tamanioSensor)
    
    ## Descomponemos M en Ma, Mb, Mc, que son sus filas
    ma = matCalib[0,:]
    mb = matCalib[1,:]
    mc = matCalib[2,:]
    
    ma = np.array([ma])
    mb = np.array([mb])
    mc = np.array([mc])
    
    ## Calculamos las transpuestas
    ma = ma.transpose()
    mb = mb.transpose()
    mc = mc.transpose()
    
    ## primera parte de la ecuacion 
    print("primera parte")
    #mc = mc.transpose() ----- ver porque la letra dice transpuesta de la transpuesta de la matriz fila
    
    print("mc")
    print(mc)
    primeraParte = pos.dot(mc)
        
        
    ## segunda parte de la ecuacion
    print("segunda parte")
    con = np.concatenate((ma,mb), axis=1)
    print(con)
    
    #trasnpuestaConcat = con.transpose() -- ver el tema de la tanspuesta, porque ya esta transpuesta
    posXtrasnp = pos.dot(con)
    
    ## despejamos (u,v)^T
    print("sensor")
    posSensor = np.divide(posXtrasnp, primeraParte)
    posSensor = posSensor.transpose() #calculo (u,v)
    print(posSensor)
    
    
    ### Con los datos obtenidos utilizamos la ecuacion: a(u,v,1)^T = M(x,y,z,1)^T
      # despejamos alfa para evaluar la condicion alfa > 0
      
    # Primera parte de la ecuacion
    sensorMasUno = posSensor.transpose()
    sensorMasUno = np.append(sensorMasUno,np.ones([len(sensorMasUno),1]),1) 
    sensorT = sensorMasUno.transpose()
    print("sensor-primera parte")
    print(sensorT)
    
    # Segunda parte de la ecuacion
    posTranspuesta = pos.transpose()
    posicionMasUnos = np.append(posTranspuesta,np.ones([len(posTranspuesta),1]),1) 
    pTransp = posicionMasUnos.transpose()
    pPorM = matCalib.dot(pTransp)
    print("posicion - segunda parte")
    print(pPorM)
    
    
    # Despejamos alfa
    print("ALFA")
    #alfa = np.divide(pPorM,sensorT) ## ver porque no anda
    if alfa > 0:
        resultado = "ok"
    else:
        resultado = "error"
    print(alfa)
    
   # Verificamos la posicion del sensor utilizando el tamanio del sensor
    u = posSensor[0,:]
    v = posSensor[1,:]
    
    # absolutos de u, v
    absu = np.absolute(u)
    absv = np.absolute(v)
     
    
    # fila 1 y fila 2 del tama;no del sensor qx y qy
    qx = tamanioSensor[0,:]
    qy = tamanioSensor[0,:]
    
    # q = 2.k, hayamos k
    kx = qx.divide(2)
    ky = qy.divide(2)
    
    # se evalua abs(u) <= kx y abs(v) <= ky
    
    if absu >= kx and absv >= ky:
        resultado = "ok"
    else:
        resultado = "error"
        
        
    # si hay error los puntos no sirven.... (ver bien si la matriz da diferentes puntos como evaluarlos)
    if resultado == "ok":
        return posSensor
    else:
        return 0
        

    

In [77]:
m = [[0,2,1,1],[4,3,0,8],[2,0,0,4]]
p = [[1,2,1,1],[2,1,1,2],[1,2,2,2]]
ta = [[1],[2]]
proyectar(p,m,ta)

primera parte
mc
[[2]
 [0]
 [0]
 [4]]
segunda parte
[[0 4]
 [2 3]
 [1 0]
 [1 8]]
sensor
[[1 0 0]
 [3 2 2]]
sensor-primera parte
[[ 1.  0.  0.]
 [ 3.  2.  2.]
 [ 1.  1.  1.]]
posicion - segunda parte
[[  6.   5.   5.   7.]
 [ 18.  19.  15.  18.]
 [  6.   8.   6.   6.]]
ALFA


NameError: global name 'alfa' is not defined